In [12]:
#Caso homogeneo como validacion
using LinearAlgebra
using DifferentialEquations

function g!(dg, g, p, t)
    q0, qx = p
    q_aux = sqrt(qx^2 + g[3]^2)
    dg[1] = 340*qx/q_aux
    dg[2] = 340*g[3]/q_aux
    dg[3]=0
end
Source=[0.,100.]
Receiver=[1000.,100.]
# Con la fuente y el receptor damos un ángulo inicial.
q0,q_x,q_z,r,theta_i=Slowness_Spherical_2D(Source,Receiver, 340.)
##########
#Medimos el tiempo que le llevaria  a una recta
Delta_x=Receiver[1]-Source[1]
Delta_t=Delta_x/(340*q_x/q0)
thetas= (theta_i-1):0.01:(theta_i+1)
###########
# Primero trayectoria y luego como varia qz con t
angulos = []
tiempos = []
for i in 1:length(thetas)
    q_x=q0*sind(thetas[i])
    q_z=q0*cosd(thetas[i])
    # Definir los parámetros
    p = [q0, q_x]
    u0 = [Source[1],Source[2], q_z]
    tspan=[0,Delta_t+1]
    # Crear el problema ODE
    prob = ODEProblem(g!, u0, tspan, p)
    # Resolver el problema ODE
    sol = solve(prob, RK4(), dt=1e-6, saveat=0:0.00001:Delta_t+1)
    # Comprobar si alguna solución está cerca del receptor
    for j in 1:length(sol.u)
        if abs(sol.u[j][1] - Receiver[1]) < 1 && abs(sol.u[j][2] - Receiver[2]) < 1
            push!(angulos, thetas[i])
            push!(tiempos, sol.t[j])
            println("thetas[$i] = ", thetas[i], ", tiempo = ", sol.t[j])
            break
        end
    end
end

thetas[96] = 89.95, tiempo = 2.93824
thetas[97] = 89.96, tiempo = 2.93824
thetas[98] = 89.97, tiempo = 2.93824
thetas[99] = 89.98, tiempo = 2.93824
thetas[100] = 89.99, tiempo = 2.93824
thetas[101] = 90.0, tiempo = 2.93824
thetas[102] = 90.01, tiempo = 2.93824
thetas[103] = 90.02, tiempo = 2.93824
thetas[104] = 90.03, tiempo = 2.93824
thetas[105] = 90.04, tiempo = 2.93824
thetas[106] = 90.05, tiempo = 2.93824


In [1]:
using LinearAlgebra
function Analytical_RP(theta,alpha_c,c0,z)
    psi=cosd(theta)/c0 #Coseno para ángulos en grados
    x=(sqrt(1-(psi*c0)^2)-sqrt(1-psi^2*(c0+alpha_c*(z-SourcePoint[2]))^2))/(psi*alpha_c)
    return x
end
SourcePoint=[0.,100.]
alpha_c=0.6 ;
c0=332+0.6*100;
using Roots
@time begin
theta1=75
x1_coords=0.:1:60.
z1_coords=[]
for i in 1:length(x1_coords)
    function objective_function(z)
        x_target = x1_coords[i]  # reemplaza esto con x dado
        x_function = Analytical_RP(theta1, alpha_c, c0, z)
        return x_function - x_target
    end
    z_initial_guess = SourcePoint[2]# Suposición inicial para z
    z_solution = fzero(objective_function, z_initial_guess)
    push!(z1_coords,z_solution)
end
#####################
theta2=45
x2_coords=0.:1:150.
z2_coords=[]
for i in 1:length(x2_coords)
    function objective_function(z)
        x_target = x2_coords[i]  # reemplaza esto con x dado
        x_function = Analytical_RP(theta2, alpha_c, c0, z)
        return x_function - x_target
    end
    z_initial_guess = SourcePoint[2]# Suposición inicial para z
    z_solution = fzero(objective_function, z_initial_guess)
    push!(z2_coords,z_solution)
end
###########################
theta3=15
x3_coords=0.:1:86.
z3_coords=[]
for i in 1:length(x3_coords)
    function objective_function(z)
        x_target = x3_coords[i]  # reemplaza esto con x dado
        x_function = Analytical_RP(theta3, alpha_c, c0, z)
        return x_function - x_target
    end
    z_initial_guess = SourcePoint[2]# Suposición inicial para z
    z_solution = fzero(objective_function, z_initial_guess)
    push!(z3_coords,z_solution)
end
end

  2.828059 seconds (3.78 M allocations: 247.691 MiB, 3.78% gc time, 98.15% compilation time)


# Iterar barrido angulos


In [6]:
Source=[0.,100.]
c_source=c_z(100)
Receiver=[x1_coords[end],z1_coords[end]]
c_receiver=c_z(z1_coords[end])
# Con la fuente y el receptor damos un ángulo inicial.
q0,q_x,q_z,r,theta_i=Slowness_Spherical_2D(Source,Receiver, c_source)
println(theta_i)
##########
#Medimos el tiempo que le llevaria  a una recta
Delta_x=Receiver[1]-Source[1]
c_m=(c_source+c_receiver)/2
Delta_t=Delta_x/(c_m*q_x/q0)
for k in 1:15
    thetas= range(theta_i-45/(2^(k-1)), theta_i+45/(2^(k-1)), 100); 
    ###########
    angulos = []
    tiempos = []
    for i in 1:length(thetas)
        q_x=q0*sind(thetas[i])
        q_z=q0*cosd(thetas[i])
        # Definir los parámetros
        p = [q0, q_x]
        u0 = [Source[1],Source[2], q_z]
        tspan=[0,Delta_t+1] #Tiempo lo suficientemente grande
        # Crear el problema ODE
        prob = ODEProblem(g!, u0, tspan, p)
        # Resolver el problema ODE
        sol = solve(prob, RK4(), dt=1e-5, saveat=0:0.0001:Delta_t+1)
        # Comprobar si alguna solución está cerca del receptor
        for j in 1:length(sol.u)
            if sqrt((sol.u[j][1] - Receiver[1])^2 +(sol.u[j][2] - Receiver[2])^2) < r/(20*2^(k-1))
                push!(angulos, thetas[i])
                push!(tiempos, sol.t[j])
                break
            end
        end
    end
    if length(tiempos)==0
        break
    else
        traveltime_minimo = findmin(tiempos)
        println("El traveltime para la iteración ", k , " es: ", traveltime_minimo[1])
        indice = argmin(tiempos)
        theta_i=angulos[indice]
        println("El ángulo  para la iteración ", k , " es: ", theta_i)
    end
end
println("--------------")
Source=[0.,100.]
c_source=c_z(100)
Receiver=[x3_coords[end],z3_coords[end]]
c_receiver=c_z(z3_coords[end])
# Con la fuente y el receptor damos un ángulo inicial.
q0,q_x,q_z,r,theta_i=Slowness_Spherical_2D(Source,Receiver, c_source)
println(theta_i)
##########
#Medimos el tiempo que le llevaria  a una recta
Delta_x=Receiver[1]-Source[1]
c_m=(c_source+c_receiver)/2
Delta_t=Delta_x/(c_m*q_x/q0)
for k in 1:15
    thetas= range(theta_i-45/(2^(k-1)), theta_i+45/(2^(k-1)), 100); 
    ###########
    angulos = []
    tiempos = []
    for i in 1:length(thetas)
        q_x=q0*sind(thetas[i])
        q_z=q0*cosd(thetas[i])
        # Definir los parámetros
        p = [q0, q_x]
        u0 = [Source[1],Source[2], q_z]
        tspan=[0,Delta_t+1] #Tiempo lo suficientemente grande
        # Crear el problema ODE
        prob = ODEProblem(g!, u0, tspan, p)
        # Resolver el problema ODE
        sol = solve(prob, RK4(), dt=1e-5, saveat=0:0.0001:Delta_t+1)
        # Comprobar si alguna solución está cerca del receptor
        for j in 1:length(sol.u)
            if sqrt((sol.u[j][1] - Receiver[1])^2 +(sol.u[j][2] - Receiver[2])^2) < r/(20*2^(k-1))
                push!(angulos, thetas[i])
                push!(tiempos, sol.t[j])
                break
            end
        end
    end
    if length(tiempos)==0
        break
    else
        traveltime_minimo = findmin(tiempos)
        println("El traveltime para la iteración ", k , " es: ", traveltime_minimo[1])
        indice = argmin(tiempos)
        theta_i=angulos[indice]
        println("El ángulo  para la iteración ", k , " es: ", theta_i)
    end
end

17.29151807676903
El traveltime para la iteración 1 es: 0.4312
El ángulo  para la iteración 1 es: 15.018790804041757
El traveltime para la iteración 2 es: 0.4414
El ángulo  para la iteración 2 es: 14.79151807676903
El traveltime para la iteración 3 es: 0.44630000000000003
El ángulo  para la iteración 3 es: 14.905154440405394
El traveltime para la iteración 4 es: 0.44870000000000004
El ángulo  para la iteración 4 es: 14.961972622223575
El traveltime para la iteración 5 es: 0.4499
El ángulo  para la iteración 5 es: 14.990381713132667
El traveltime para la iteración 6 es: 0.4506
El ángulo  para la iteración 6 es: 14.976177167678122
El traveltime para la iteración 7 es: 0.4509
El ángulo  para la iteración 7 es: 14.983279440405395
El traveltime para la iteración 8 es: 0.451
El ángulo  para la iteración 8 es: 14.993932849496304
El traveltime para la iteración 9 es: 0.4511
El ángulo  para la iteración 9 es: 14.992157281314485
El traveltime para la iteración 10 es: 0.4511
El ángulo  para la it

In [7]:
Receiver=[x3_coords[end],z3_coords[end]]

2-element Vector{Float64}:
  86.0
 117.15811952934443

In [3]:
using LinearAlgebra
using DifferentialEquations
@time begin
function Slowness_Spherical_2D(Point1::Array{Float64, 1}, Point2::Array{Float64, 1}, c_source)
    q0=1/c_source #Habría que variarlo si hay viento
    x1,z1=Point2-Point1
    r=norm(Point2-Point1)
    theta=rad2deg(acos(z1/r)) 
    qx=abs(q0*sind(theta))
    qz=q0*cosd(theta)
    return q0,qx,qz,r,theta
end
function c_z(z)
    c=332+0.6*z
    return c
end
function g!(dg, g, p, t)
    q0, qx = p
    q_aux = sqrt(qx^2 + g[3]^2)
    dg[1] = c_z(g[2])*qx/q_aux #Variacion coordenada X
    dg[2] = c_z(g[2])*g[3]/q_aux #Variacion coordenada Z
    dg[3]=-q_aux*0.6 #Variacion qz
end
Source=[0.,100.]
c_source=c_z(100)
Receiver=[150.,221.4856419405331]
# Con la fuente y el receptor damos un ángulo inicial.
q0,q_x,q_z,r,theta_i=Slowness_Spherical_2D(Source,Receiver, c_source)
println(theta_i)
##########
#Medimos el tiempo que le llevaria  a una recta a la velocidad de sonido mínima en la región
Delta_t=r/c_source
for k in 1:15
    thetas= range(theta_i-45/(2^(k-1)), theta_i+45/(2^(k-1)), 100); 
    ###########
    angulos = []
    tiempos = []
    for i in 1:length(thetas)
        q_x=q0*sind(thetas[i])
        q_z=q0*cosd(thetas[i])
        # Definir los parámetros
        p = [q0, q_x]
        u0 = [Source[1],Source[2], q_z]
        tspan=[0,Delta_t] #Tiempo lo suficientemente grande
        # Crear el problema ODE
        prob = ODEProblem(g!, u0, tspan, p)
        # Resolver el problema ODE
        sol = solve(prob, RK4(), dt=1e-5, saveat=0:0.0001:Delta_t)
        # Comprobar si alguna solución está cerca del receptor
        for j in 1:length(sol.u)
            if sqrt((sol.u[j][1] - Receiver[1])^2 +(sol.u[j][2] - Receiver[2])^2) < r/(20*2^(k-1))
                push!(angulos, thetas[i])
                push!(tiempos, sol.t[j])
                break
            end
        end
    end
    if length(tiempos)==0
        break
    else
        traveltime_minimo = findmin(tiempos)
        println("El traveltime para la iteración ", k , " es: ", traveltime_minimo[1])
        indice = argmin(tiempos)
        theta_i=angulos[indice]
        println("El ángulo  para la iteración ", k , " es: ", theta_i)
    end
end
end

50.995839223434295
El traveltime para la iteración 1 es: 0.43
El ángulo  para la iteración 1 es: 45.08674831434339
El traveltime para la iteración 2 es: 0.4405
El ángulo  para la iteración 2 es: 44.85947558707066
El traveltime para la iteración 3 es: 0.4456
El ángulo  para la iteración 3 es: 44.97311195070703
El traveltime para la iteración 4 es: 0.44830000000000003
El ángulo  para la iteración 4 es: 44.91629376888885
El traveltime para la iteración 5 es: 0.4495
El ángulo  para la iteración 5 es: 45.00152104161612
El traveltime para la iteración 6 es: 0.45020000000000004
El ángulo  para la iteración 6 es: 44.987316496161576
El traveltime para la iteración 7 es: 0.4505
El ángulo  para la iteración 7 es: 44.99441876888885
El traveltime para la iteración 8 es: 0.45070000000000005
El ángulo  para la iteración 8 es: 44.98376535979794
El traveltime para la iteración 9 es: 0.45080000000000003
El ángulo  para la iteración 9 es: 44.992643200707036
El traveltime para la iteración 10 es: 0.450800

# Mc Bride

In [9]:
using Plots
function g!(dg, g, p, t)
    q0, qx = p
    q_aux = sqrt(qx^2 + g[3]^2)
    dg[1] = c_z(g[2])*qx/q_aux #Variacion coordenada X
    dg[2] = c_z(g[2])*g[3]/q_aux #Variacion coordenada Z
    dg[3]=-q_aux*0.6 #Variacion qz
end
#Emision a 75 grados
q_x=q0*sind(14.998371769950849)
q_z=q0*cosd(14.998371769950849)
# Definir los parámetros
p = [q0, q_x]
u0 = [Source[1],Source[2], q_z]
tspan=[0,0.4511]
# Crear el problema ODE
prob = ODEProblem(g!, u0, tspan, p)
# Resolver el problema ODE
sol = solve(prob, RK4(), dt=1e-6, saveat=0:0.00001:0.4511)
# Extraer los datos de la solución
x_data_75 = [u[1] for u in sol.u]
z_data_75 = [u[2] for u in sol.u]

#Emision a 45 grados
q_x=q0*sind(44.99841379729794)
q_z=q0*cosd(44.99841379729794)
# Definir los parámetros
p = [q0, q_x]
u0 = [Source[1],Source[2], q_z]
tspan=[0,0.45080000000000003]
# Crear el problema ODE
prob = ODEProblem(g!, u0, tspan, p)
# Resolver el problema ODE
sol = solve(prob, RK4(), dt=1e-6, saveat=0:0.00001:0.45080000000000003)
# Extraer los datos de la solución
x_data_45 = [u[1] for u in sol.u]
z_data_45 = [u[2] for u in sol.u]

#Emision a 15 grados
q_x=q0*sind(74.99353243040824)
q_z=q0*cosd(74.99353243040824)
# Definir los parámetros
p = [q0, q_x]
u0 = [Source[1],Source[2], q_z]
tspan=[0,0.2207]
# Crear el problema ODE
prob = ODEProblem(g!, u0, tspan, p)
# Resolver el problema ODE
sol = solve(prob, RK4(), dt=1e-6, saveat=0:0.00001:0.2207)
# Extraer los datos de la solución
x_data_15 = [u[1] for u in sol.u]
z_data_15 = [u[2] for u in sol.u]

# Crear la gráfica
p=plot(legend=:topright, legendfontsize=6)
plot!(x_data_75, z_data_75, xlabel="x (m)", ylabel="z (m)", label="Implementation \$\\theta\$=14.998º : \$\\tau\$ =0.4511 s ", color=:blue)
plot!(x_data_45, z_data_45,  label="Implementation \$\\theta\$=44.998º : \$\\tau\$ =0.4508 s ", color=:green)
plot!(x_data_15, z_data_15, label="Implementation \$\\theta\$=74.994º : \$\\tau\$ =0.2207 s ", color=:red)
plot!(x3_coords,z3_coords, label="Analytical Ray Path \$\\theta\$=15º : \$\\tau\$ =0.4511 s ", color=:black, style=:dash)
plot!(x2_coords,z2_coords, label="Analytical Ray Path \$\\theta\$=45º : \$\\tau\$ =0.4507 s ", color=:black, style=:dash)
plot!(x1_coords,z1_coords, label="Analytical Ray Path \$\\theta\$=75º : \$\\tau\$ =0.2207 s ", color=:black, style=:dash)

savefig(p, "mcbride(barridoangulos)_temporalgradient.png")

"C:\\Users\\Miguel\\Desktop\\TFM\\Codigos Julia\\mcbride(barridoangulos)_temporalgradient.png"